# Modelo de Sistema de Recomendação do Me Empresta

No mundo moderno sistemas de aprendizagem de máquino promovem um forma inteligente de lidar com dados e tomadas de decisões. Nesse sentido, procuramos através deste documentos elaborar um modelo de inteligência artificial baseando-se em sistemas de recomendação colaborativos.


### Filtragem colaborativa

- A filtragem colaborativa é baseada na noção que pessoas curtem coisas parecidas que outras pessoas parecidas com elas curtem também.

# Colunas utilizadas:

● customer_id : Todo o usuário cadastrado com um id único

● product_id : Todo produto cadastrado com um id único

● review_score : Classificação de todo o produto relacionado com o usuário 

● timestamp : Horário de cadastro



# Importação de bibliotecas

In [ ]:
import numpy as np # manipulação de arrays e algebra linear
import pandas as pd # Processamento de dados e leitura de CSV
import os
import gc
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Carregando o Dataset e renomeando as colunas

In [ ]:
electronics_data=pd.read_csv("/kaggle/input/amazon-product-reviews/ratings_Electronics (1).csv",
                             names=['user_id', 'product_id','rating','timestamp'])

# Apresentação dos dados

In [ ]:
electronics_data.head()

# Informações do Dataset

- Quantidade de linhas e colunas do Dataset

In [ ]:
electronics_data.shape

- Tipos de dados de cada coluna

In [ ]:
electronics_data.dtypes

# Quantidade de produtos por faixa de avaliação

In [ ]:
with sns.axes_style('white'):
    g = sns.factorplot("rating", data=electronics_data, aspect=2.0,kind='count')
    g.set_ylabels("Total number of ratings")

# Removendo a coluna de timestamp

In [ ]:
electronics_data.drop(['timestamp'], axis=1, inplace=True)

electronics_data.head()

# Filtragem colaborativa (Item- Item)

In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split

### Agrupando o dataset por produtos que tem uma avaliação mais ou igual a 50

In [ ]:
electronics_data_group = electronics_data.groupby("product_id").filter(lambda x:x['rating'].count() >=50)

electronics_data_group.shape

Fatiar o Dataset para lidar com uma quantidade reduzida de dados

In [ ]:
electronics_data_group = electronics_data_group[0:100000]

### Leitura do Dataset utilizando o Surprise

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(electronics_data_group,reader)

### Separação entre dataset de treino e de teste

In [ ]:
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

Treinamento do algoritmo de treino com base em item - item 

In [ ]:
algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Rodando o modelo de treinamento contra o de teste

In [ ]:
test_pred = algo.test(testset)

test_pred

## Mostra a acuracia o RMSE

In [ ]:
accuracy.rmse(test_pred, verbose=True)

# Baseado em modelo user - item

### Criação de uma tabela que relaciona product_id e user_id

In [ ]:

group=electronics_data.groupby("product_id").filter(lambda x:x['rating'].count() >=50)

group_reduce=group.head(10000)

ratings_matrix = group_reduce.pivot_table(values='rating', index='user_id', columns='product_id', fill_value=0)
ratings_matrix.head()

Invertendo a tabela

In [ ]:
X = ratings_matrix.T
X.head()

In [ ]:
X1 = X

Truncar para apenas 10 colunas

In [ ]:
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

Fazer correlação

In [ ]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

In [ ]:
X.index[75]

### Buscar um produto específico

In [ ]:
i = "B00000K135"

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

### Buscar a correlação

In [ ]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape


## Recomendando o top 25 produtos em sequencia

In [ ]:
Recommend = list(X.index[correlation_product_ID > 0.65])

Recommend.remove(i) 

Recommend[0:24]

# Trabalhos Futuros

- Filtragem baseada em conteúdo, pois todos os produtos possuem uma descrição